首先請使用者輸入股票價格、執行價格(strike price)、無風險利率、期數

In [1]:
import math

s = int(input('Stock Price: '))
x = int(input('Strike Price: '))
r = float(input('Riskless Interest Rate(%): '))/100
T = int(input('Time Periods: ')) + 1

Stock Price: 160
Strike Price: 150
Riskless Interest Rate(%): 18.232
Time Periods: 3


計算債券獲利(R)以及給定股票上漲下跌幅度50%，即可計算風險中性機率。
為了以防機率為負或大於1，我寫了條件子句保證機率介於0和1之間。

In [2]:
R = math.exp(r)
R2 = math.exp(-r)
u = 1.5
d = 0.5
p = (R - d)/(u - d) #risk-neutral probability
if p < 0:
    p = 0
if p > 1:
    p = 1

設一個買權(call)的二維陣列，利用一個雙層迴圈可以依序計算未來股票價格並儲存到其中。由於我的陣列維度都相同，陣列第一層只有第一項有值，第二層只有前兩項有值，依此類推(可以參考我印出的call結果)。

In [3]:
call = [[0]*T for i in range(T)]
for i in range(T):
    for j in range(i+1):
        call[i][j] = s * u**(i-j) * d**j
print(call)

[[160.0, 0, 0, 0], [240.0, 80.0, 0, 0], [360.0, 120.0, 40.0, 0], [540.0, 180.0, 60.0, 20.0]]


有了每期的股票價格，就可以用倒推求解現在的選擇權價格。從最後一期倒推時我們要在報酬跟0之間取大的值，往前推算就可以用投影片中的公式，依序把後一期的兩個股票價格以機率加權反算，一直到現在的價格，最後印出陣列第一層的第一個值即為我們要的選擇權價格。

In [4]:
f = [[0]*T for i in range(T)]
for i in range(T):
    for j in range(T):
        if i == 0:
            f[T-i-1][j] = max(call[T-i-1][j] - x, 0)
        elif j+1 <= T-1:
            f[T-i-1][j] = (p*f[T-i][j] + (1-p)*f[T-i][j+1])*R2

print(f[0][0])

85.06922876361149
